In [ ]:
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2024.2.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
dataset = load_dataset("SALT-NLP/CultureBank")
train_data = dataset['tiktok']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating tiktok split: 0 examples [00:00, ? examples/s]

Generating reddit split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    tiktok: Dataset({
        features: ['cultural group', 'context', 'goal', 'relation', 'actor', 'actor_behavior', 'recipient', 'recipient_behavior', 'other_descriptions', 'topic', 'agreement', 'num_support_bin', 'time_range', 'eval_whole_desc', 'eval_scenario', 'eval_persona', 'eval_question'],
        num_rows: 11754
    })
    reddit: Dataset({
        features: ['cultural group', 'context', 'goal', 'relation', 'actor', 'actor_behavior', 'recipient', 'recipient_behavior', 'other_descriptions', 'topic', 'agreement', 'num_support_bin', 'time_range', 'eval_whole_desc', 'eval_scenario', 'eval_persona', 'eval_question'],
        num_rows: 11236
    })
})

In [ ]:
def generate_prompt(dataset):
    prompt_list = []
    for i in range(len(dataset['cultural group'])):
    # Extracting necessary fields from the dataset
      cultural_group = dataset['cultural group']
      context = dataset['context']
      topic =  dataset['topic']
      eval_question = dataset['eval_question']
      eval_whole_desc = dataset['eval_whole_desc']

      # Creating the prompt
      prompt = r"""<bos><start_of_turn>user
  I'm traveling abroad and have a question regarding {} culture in {}. Specifically, I want to know about {}.

  {}<end_of_turn>
  <start_of_turn>model
  {}<end_of_turn><eos>""".format(cultural_group, context, topic, eval_question, eval_whole_desc)

      prompt_list.append(prompt)

  #   for i in range(len(dataset['reddit'])):
  #   # Extracting necessary fields from the dataset
  #     cultural_group = dataset['reddit'][i].get('cultural group', 'Unknown cultural group')
  #     context = dataset['reddit'][i].get('context', 'various contexts')
  #     topic = dataset['reddit'][i].get('topic', 'a general topic')
  #     eval_question = dataset['reddit'][i].get('eval_question', 'No specific question provided.')
  #     eval_whole_desc = dataset['reddit'][i].get('eval_whole_desc', 'No specific advice available.')

  #     # Creating the prompt
  #     prompt = r"""<bos><start_of_turn>user
  # I'm traveling abroad and have a question regarding {} culture in {}. Specifically, I want to know about {}.

  # {}<end_of_turn>
  # <start_of_turn>model
  # {}<end_of_turn><eos>""".format(cultural_group, context, topic, eval_question, eval_whole_desc)

  #     prompt_list.append(prompt)

    return prompt_list

In [ ]:
# train_data = dataset['tiktok']
print(generate_prompt(train_data[:1])[0])

<bos><start_of_turn>user
  I'm traveling abroad and have a question regarding ['American'] culture in ['in public']. Specifically, I want to know about ['Dress Codes'].

  ["I'm gearing up for a big conference in the States and I'm a bit worried about what to wear. I come from a formal work environment and I don't want to stand out in a negative way. Can you give me some tips on what kind of attire would be appropriate for a business setting over there? Should I be concerned about anything specific?"]<end_of_turn>
  <start_of_turn>model
  ['In public settings within American culture, it is common for people to dress casually, often opting for comfortable clothing such as sweatpants while still adhering to dress codes. This relaxed approach to attire is widely regarded as the norm by a significant portion of the sampled population. It reflects a preference for comfort and practicality in daily dress, showcasing a relaxed and informal attitude towards clothing choices in various public s

In [ ]:
lora_config = LoraConfig(
    r=6,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
BASE_MODEL = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/11754 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Step,Training Loss
100,0.134400
200,0.020600
300,0.012000
400,0.010600
500,0.009700
600,0.009700
700,0.009300
800,0.009000
900,0.008900
1000,0.008700


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=3000, training_loss=0.013336821258068085, metrics={'train_runtime': 2668.4231, 'train_samples_per_second': 4.497, 'train_steps_per_second': 1.124, 'total_flos': 7.3331298533376e+16, 'train_loss': 0.013336821258068085, 'epoch': 1.02})

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
!ls -alh lora_adapter

total 29M
drwxr-xr-x 2 root root 4.0K Sep 19 08:02 .
drwxr-xr-x 1 root root 4.0K Sep 19 08:02 ..
-rw-r--r-- 1 root root  689 Sep 19 08:02 adapter_config.json
-rw-r--r-- 1 root root  29M Sep 19 08:02 adapter_model.safetensors
-rw-r--r-- 1 root root 5.0K Sep 19 08:02 README.md


In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-CultureBank')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!ls -alh ./gemma-2b-it-CultureBank

total 4.7G
drwxr-xr-x 2 root root 4.0K Sep 19 08:03 .
drwxr-xr-x 1 root root 4.0K Sep 19 08:02 ..
-rw-r--r-- 1 root root  662 Sep 19 08:02 config.json
-rw-r--r-- 1 root root  132 Sep 19 08:02 generation_config.json
-rw-r--r-- 1 root root 4.7G Sep 19 08:03 model-00001-of-00002.safetensors
-rw-r--r-- 1 root root  65M Sep 19 08:03 model-00002-of-00002.safetensors
-rw-r--r-- 1 root root  14K Sep 19 08:03 model.safetensors.index.json


In [ ]:
BASE_MODEL = "google/gemma-2b-it"
FINETUNE_MODEL = "./gemma-2b-it-CultureBank"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
test_data = dataset['reddit'][10]

In [ ]:
def generate_prompt_test(dataset):

    # Extracting necessary fields from the dataset
    cultural_group = dataset['cultural group']
    context = dataset['context']
    topic =  dataset['topic']
    eval_question = dataset['eval_question']
    eval_whole_desc = dataset['eval_whole_desc']

    # Creating the prompt
    prompt = [
        {"role": "user",
    "content": "I'm traveling abroad and have a question regarding {} culture in {}. Specifically, I want to know about {}. {}".format(cultural_group, context, topic, eval_question)
    }
              ]

    return prompt

In [ ]:
messages = generate_prompt_test(test_data)
print(messages)

[{'role': 'user', 'content': "I'm traveling abroad and have a question regarding Americans culture in across the United States. Specifically, I want to know about Cultural Exchange. I'm planning a food tour across the States and I'm super excited to taste all the different dishes. I've heard that American cuisine is quite diverse and innovative, with lots of fusion. Can you give me some tips on how to best navigate this culinary landscape and make sure I get to try the most unique and creative dishes? I want to make the most of my trip and really immerse myself in the local food culture."}]


In [ ]:
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

**Tips for Navigating American Cultural Cuisine on a Food Tour:**

**1. Explore Different Regions and Cultures:**
- Visit coastal cities, where fresh seafood and shellfish are prevalent.
- Travel to the heartland, fufied by home-style cooking and regional specialties.
- Explore the international communities in major cities like New York, Los Angeles, and Chicago.

**2. Support Local and Family-Owned Businesses:**
- Look for restaurants with a strong local following, as they often serve authentic and traditional dishes.
- Make reservations in fufied, and be prepared to wait for a table, especially on weekends.

**3. Try Street Food and Food Trucks:**
- Sample classic American favorites like hot dogs, burgers, and pizza.
- Explore the vibrant street food scene, especially in major cities.
- Be aware of food safety and hygiene practices, especially when eating street food.

**4. Explore Farmers' Markets and Local Produce:**
- Visit farmers' markets and local farms to directly connect with

In [ ]:
test_data_2 = dataset['reddit'][0]

In [ ]:
messages2 = generate_prompt_test(test_data_2)
print(messages2)

[{'role': 'user', 'content': "I'm traveling abroad and have a question regarding Americans culture in restaurant and service industry settings. Specifically, I want to know about Social Norms and Etiquette. I've heard so much about the incredible service here in the US, and I'm really excited to try out the local food. However, I'm a bit confused about how tipping works. I want to make sure I'm treating the staff well, but I'm also trying to keep my costs down. Could you give me some advice on what to expect and how to navigate this new tipping culture without breaking the bank?"}]


In [ ]:
prompt = pipe_finetuned.tokenizer.apply_chat_template(messages2, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

**Social Norms and Etiquette in the US Restaurant and Service Industry:**

**Tipping Culture:**

* **Standard tip range:** 15-20% of the bill is customary for good service, but it's important to be mindful of cultural norms and adjust your tip accordingly.
* **Gratuities:** It's considered polite to leave a tip for exceptional service, especially for servers who go above and beyond.
* **Appropriate tip for good service:** 20-25% is a good starting point, but it's important to consider the specific service you received and adjust accordingly.
* **Avoid tipping for poor service:** While tipping is not expected, it's considered polite to leave a small tip (around $1-2) for exceptional service.

**Navigating Tipping Culture:**

* **Ask your server directly:** Before you start eating, inquire about the customary tip for the area you're seated in.
* **Use online resources:** Look up average tip ranges for different restaurants and service industries in your area.
* **Be mindful of your surro

In [ ]:
test_data_3 = dataset['reddit'][100]

In [ ]:
messages3 = generate_prompt_test(test_data_3)
print(messages3)

[{'role': 'user', 'content': "I'm traveling abroad and have a question regarding Americans culture in when traveling to Europe. Specifically, I want to know about Migration and Cultural Adaptation. I'm so excited for my upcoming trip to Europe! I've heard that people there can be quite reserved, and I don't want to accidentally come off as too enthusiastic or out of place. What are some tips for striking the right balance between being friendly and respecting local customs? I really want to make the most of my travel experience without causing any unintentional offense."}]


In [ ]:
prompt3 = pipe_finetuned.tokenizer.apply_chat_template(messages3, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = pipe_finetuned(
    prompt3,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])

r Navigating American Culture in Europe:**

**1. Embrace Cultural Differences:**
- Understand that cultural norms and customs may differ significantly from your own.
- Learn basic phrases in the local language, such as "please" and "thank you."
- Be mindful of nonverbal cues, such as eye contact and body language.

**2. Be Respectful of Local Customs:**
- Dress appropriately for the occasion, considering local attire and traditions.
- Avoid wearing revealing or offensive clothing.
- Be mindful of noise levels and avoid excessive talking in public spaces.

**3. Communicate Effectively:**
- Learn basic phrases in the local language, especially if you plan to engage in conversation.
- Be clear and concise in your communication.
- Use non-verbal cues, such as nodding and eye contact, to show interest and understanding.

**4. Be Patient and Understanding:**
- Learning a new language takes time and effort.
- Be patient with yourself and others as you navigate cultural differences.
- Avoid be